In [1]:
import pandas as pd
import os
import boto3
import numpy as  np
import pandas as pd
import json
import time
from multiprocessing.dummy import Pool

In [2]:
os.chdir('//FS2\RTO\\CIS-PD MUSC\\')
client = boto3.client('athena', region_name = 'us-west-2')
user_id_pairings = pd.read_csv("decoded_forms\\videoID.csv")
subjenroll = pd.read_sas("subjenrollment.sas7bdat")
startstop = pd.read_csv("decoded_forms\\WatchStartStop.csv")

# set path to folder containing subject records
path = r'//FS2.smpp.local\\RTO\\CIS-PD Study\MJFF Curation\Finalized Dataset'
# set path to destination folder
dest = r'//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Record Correlation'
#---------------------------------------------------------------------------------------------------------

In [3]:
# Medication Reports
table_med = 'Table10.csv'
dest_ext_med = 'Medication Reports'
file_name_med = 'med_timepoints.csv'

# Symptom Reports
table_symt = 'Table11.csv'
dest_ext_symt = 'Symptom Reports'
file_name_symt = 'symt_timepoints.csv'

# Diaries
table_diar = 'Table12.csv'
dest_ext_diar = 'Diaries'
file_name_diar = 'diar_timepoints.csv'

# Load Consolidated DataFrames
timepoints_med = pd.read_csv(os.path.join(dest, dest_ext_med, file_name_med), parse_dates = [1])
timepoints_symt = pd.read_csv(os.path.join(dest, dest_ext_symt, file_name_symt), parse_dates = [1])
timepoints_diar = pd.read_csv(os.path.join(dest, dest_ext_diar, file_name_diar), parse_dates = [1])

In [4]:
def user_id_4_to_6(user_id_4):
    
    # iterate through the rows of 'videoID.csv' file in 'decoded_forms' directory of 'CIS-PD MUSC' directory
    # file contains a column for both 4-digit and 6-digit subject ID
    for index, row in user_id_pairings.iterrows():
        if (int(row['SubjectCode']) == int(user_id_4)):
            
            # return 'None' if the SubjectCode (4-digit) has no corresponding 6-digit ID (Subj ID Athena cell in csv is empty)
            if np.isnan(row['Subj ID Athena']):
                return None
            
            # if a 6-digit code is contained within the row that has the 4-digit code, return it
            return int(row['Subj ID Athena'])
        
    # 'None' is returned if no row contains the 4-digit code
    return None

In [5]:
def generate_query(StartTimestamp, EndTimestamp, S6, S):
    template1 = "SELECT user_id-" + str(int(S6) - int(S)) + " AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z\n"
    template2 = "FROM watch_accelerometer\nwhere cohort != 'test'\nand user_id = "
    template3 = "\nand timestamp >= timestamp '"
    template4 = "'\nand timestamp < timestamp '"
    template5 = "'"
    
    startTime = str(StartTimestamp)
    endTime = str(EndTimestamp)
    
    query = template1 + template2 + S6 + template3 + startTime + template4 + endTime + template5
    
    # generate query from row in form509
    return query

In [6]:
def getTimestampsPaths(timepoints_med, timepoints_symt, timepoints_diar):
    
    # Initialize Empty Lists for Each Necessary Piece of Information from Each Record
    StartTimestamps = []
    EndTimestamps = []
    SaveFilePaths = []
    
    
    StartTimes = timepoints_med.apply(lambda row: row.Timestamp + pd.Timedelta(unit = 'minute', value = -30), axis = 1)
    EndTimes = timepoints_med.apply(lambda row: row.Timestamp + pd.Timedelta(unit = 'minute', value = 30), axis = 1)
    SavePaths = timepoints_med.apply(lambda row: os.path.join(dest, dest_ext_med, str(row.SubjID), 
                                                                                  str(row.Timestamp)[:7], 
                                                                                  str(row.Timestamp)[8:10],
                                                                                  str(row.Timestamp)[11:13] +
                                                                                  str(row.Timestamp)[14:16] +
                                                                                  str(row.Timestamp)[17:] + '.csv'), axis = 1)
    StartTimestamps = StartTimestamps + list(StartTimes)
    EndTimestamps = EndTimestamps + list(EndTimes)
    SaveFilePaths = SaveFilePaths + list(SavePaths)
    
    StartTimes = timepoints_symt.apply(lambda row: row.Timestamp + pd.Timedelta(unit = 'minute', value = -30), axis = 1)
    EndTimes = timepoints_symt.apply(lambda row: row.Timestamp, axis = 1)
    SavePaths = timepoints_symt.apply(lambda row: os.path.join(dest, dest_ext_symt, str(row.SubjID), 
                                                                                   str(row.Timestamp)[:7], 
                                                                                   str(row.Timestamp)[8:10],
                                                                                   str(row.Timestamp)[11:13] +
                                                                                   str(row.Timestamp)[14:16] +
                                                                                   str(row.Timestamp)[17:] + '.csv'),axis = 1)
    StartTimestamps = StartTimestamps + list(StartTimes)
    EndTimestamps = EndTimestamps + list(EndTimes)
    SaveFilePaths = SaveFilePaths + list(SavePaths)
    
    StartTimes = timepoints_diar.apply(lambda row: row.Timestamp + pd.Timedelta(unit = 'minute', value = -30), axis = 1)
    EndTimes = timepoints_diar.apply(lambda row: row.Timestamp, axis = 1)
    SavePaths = timepoints_diar.apply(lambda row: os.path.join(dest, dest_ext_diar, str(row.SubjID), 
                                                                                   str(row.Timestamp)[:7], 
                                                                                   str(row.Timestamp)[8:10],
                                                                                   str(row.Timestamp)[11:13] +
                                                                                   str(row.Timestamp)[14:16] +
                                                                                   str(row.Timestamp)[17:] + '.csv'),axis = 1)
    StartTimestamps = StartTimestamps + list(StartTimes)
    EndTimestamps = EndTimestamps + list(EndTimes)
    SaveFilePaths = SaveFilePaths + list(SavePaths)
    
    return StartTimestamps, EndTimestamps, SaveFilePaths

In [9]:
def iterate_reports(timepoints_med, timepoints_symt, timepoints_diar):
    
    StartTimestamps, EndTimestamps, SaveFilePaths = getTimestampsPaths(timepoints_med, timepoints_symt, timepoints_diar)
    
    # there is a new query (and also destination file paths) required for each patient report
    for s, e, p in zip(StartTimestamps, EndTimestamps, SaveFilePaths):
        
        # print the file path designation, beginning with the report type (including SubjID and full report time designation)
        print('          ' + p[63:])
        
        # get the 6-digit subject ID based on the text within the file path for each report
        if p[:81] == '//FS2.smpp.local\\\\RTO\\\\CIS-PD Study\\Patient Record Correlation\\Medication Reports':
            S = p[82:86]
            S6 = str(user_id_4_to_6(S))
        if p[:78] == '//FS2.smpp.local\\\\RTO\\\\CIS-PD Study\\Patient Record Correlation\\Symptom Reports':
            S = p[79:83]
            S6 = str(user_id_4_to_6(S))
        if p[:70] == '//FS2.smpp.local\\\\RTO\\\\CIS-PD Study\\Patient Record Correlation\\Diaries':
            S = p[71:75]
            S6 = str(user_id_4_to_6(S))
        
        # stop the function process if a 6-digit ID is not correlated (all 4-digit IDs targetted should have 6-digit IDs)
        if S6 == 'None':
            
            print('No 6-digit ID found.')
            break
            
        else:
            
            StartTimestamp = s
            EndTimestamp = e

            query = generate_query(StartTimestamp, EndTimestamp, S6, S)
            print(query)

In [10]:
# print the query for each report
iterate_reports(timepoints_med, timepoints_symt, timepoints_diar)

          Medication Reports\1004\2017-06\21\142217.csv
SELECT user_id-141575 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142579
and timestamp >= timestamp '2017-06-21 13:52:17'
and timestamp < timestamp '2017-06-21 14:52:17'
          Medication Reports\1004\2017-06\21\152858.csv
SELECT user_id-141575 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142579
and timestamp >= timestamp '2017-06-21 14:58:58'
and timestamp < timestamp '2017-06-21 15:58:58'
          Medication Reports\1004\2017-06\21\181225.csv
SELECT user_id-141575 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142579
and timestamp >= timestamp '2017-06-21 17:42:25'
and timestamp < timestamp '2017-06-21 18:42:25'
          Medication Reports\1004\2017-06\21\210434.csv
SELECT user_id-141575 AS SubjID, times

SELECT user_id-141565 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142595
and timestamp >= timestamp '2017-08-30 21:58:32.615000'
and timestamp < timestamp '2017-08-30 22:58:32.615000'
          Medication Reports\1030\2017-08\31\155328.958000.csv
SELECT user_id-141565 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142595
and timestamp >= timestamp '2017-08-31 15:23:28.958000'
and timestamp < timestamp '2017-08-31 16:23:28.958000'
          Medication Reports\1047\2017-09\06\111407.045000.csv
SELECT user_id-141539 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142586
and timestamp >= timestamp '2017-09-06 10:44:07.045000'
and timestamp < timestamp '2017-09-06 11:44:07.045000'
          Medication Reports\1047\2017-09\07\235501.636000.csv
SELECT user_id-141539 AS SubjID

SELECT user_id-141562 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142580
and timestamp >= timestamp '2017-09-11 23:00:11'
and timestamp < timestamp '2017-09-12 00:00:11'
          Medication Reports\1018\2017-09\12\111553.csv
SELECT user_id-141562 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142580
and timestamp >= timestamp '2017-09-12 10:45:53'
and timestamp < timestamp '2017-09-12 11:45:53'
          Medication Reports\1018\2017-09\14\111507.csv
SELECT user_id-141562 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142580
and timestamp >= timestamp '2017-09-14 10:45:07'
and timestamp < timestamp '2017-09-14 11:45:07'
          Medication Reports\1018\2017-09\15\111511.csv
SELECT user_id-141562 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_acc

SELECT user_id-141562 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142580
and timestamp >= timestamp '2018-01-11 12:57:42'
and timestamp < timestamp '2018-01-11 13:57:42'
          Medication Reports\1039\2017-07\19\103000.csv
SELECT user_id-141583 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142622
and timestamp >= timestamp '2017-07-19 10:00:00'
and timestamp < timestamp '2017-07-19 11:00:00'
          Medication Reports\1039\2017-07\19\171500.csv
SELECT user_id-141583 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142622
and timestamp >= timestamp '2017-07-19 16:45:00'
and timestamp < timestamp '2017-07-19 17:45:00'
          Medication Reports\1039\2017-07\19\195651.csv
SELECT user_id-141583 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_acc

SELECT user_id-141583 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142622
and timestamp >= timestamp '2017-08-06 14:00:06'
and timestamp < timestamp '2017-08-06 15:00:06'
          Medication Reports\1039\2017-08\06\185058.csv
SELECT user_id-141583 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142622
and timestamp >= timestamp '2017-08-06 18:20:58'
and timestamp < timestamp '2017-08-06 19:20:58'
          Medication Reports\1039\2017-08\06\225633.csv
SELECT user_id-141583 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_accelerometer
where cohort != 'test'
and user_id = 142622
and timestamp >= timestamp '2017-08-06 22:26:33'
and timestamp < timestamp '2017-08-06 23:26:33'
          Medication Reports\1039\2017-08\07\022956.csv
SELECT user_id-141583 AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z
FROM watch_acc

KeyboardInterrupt: 